In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt

from corner import corner
from acor import acor
from UL_uncert import UL_uncert

%matplotlib inline

In [ ]:
ephem = 'DE421'

# anomaly PSRs
psrs_anom = ['J0030+0451', 'J1012+5307', 'J1909-3744', 'J0613-0200', 'J2145-0750', 'J1918-0642']

# expected good PSRs
psrs = ['J1713+0747',  'J1643-1224', 'J1744-1134', 'B1937+21']

psr_name = psrs_anom[2]
#psr_name = psrs[3]

rundir = '/home/pbaker/nanograv/bwm/sngl/{}/{}/'.format(psr_name, ephem)

In [ ]:
# this changes depending on which "names" are used in enterprise....

with open(rundir + 'params.txt', 'r') as f:
    params = [line.rstrip('\n') for line in f]

params = params[-5:]
idx_A = params.index('log10_A_bwm')
idx_t0 = params.index('t0_bwm')

In [ ]:
chain_raw = np.loadtxt(rundir + 'chain_1.txt')
burn = int(0.1 * len(chain_raw))
chain = chain_raw[burn:]    
chain_bwm = chain[:,-9:-4]  # no ephem model

# de-log x-axis for ULs
#chain_bwm[:,idx_A] = 10**chain_bwm[:,idx_A]
#params[idx_A] = r'$A_\mathrm{BwM}$'
    
corr = acor(chain_bwm[:,idx_A])[0]
N = len(chain_bwm)
print("{:s} -- {:s}".format(psr_name,ephem))
print("N = {}, corr = {}, Neff = {}".format(N, corr, N/corr))

In [ ]:
corner_kwargs = {'bins':30,
                 'labels':params,
                 'smooth':1.0,
                 'plot_datapoints':False,
                 'plot_density':True,
                 'plot_contours':False,
                 'fill_contours':False,}

fig = corner(chain_bwm, color='C0', **corner_kwargs);

In [ ]:
ax = []
fig = plt.figure(figsize=(20,12))

for pp, par in enumerate(params):
    ax.append(fig.add_subplot(2,3,pp+1))
    ax[pp].hist(chain_bwm[:,pp], 30, normed=True, histtype='step', label=ephem)
    ax[pp].set_xlabel(par)

ax[0].legend(loc='upper left')
ax[idx_A].set_yscale('log')
ax[idx_t0].set_yscale('log')

In [ ]:
from matplotlib.colors import LogNorm
lognorm = LogNorm()

hist_kwargs = {'bins':50,
               'normed':True,
               'cmap':'YlOrRd',
               #'vmin':1.0e5,
               #'vmax':1.0e10,
               'norm':lognorm,
                }

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

t0_ch = chain_bwm[:,idx_t0]
if max(chain_bwm[:,idx_A])>0:
    h_ch = np.log10(chain_bwm[:,idx_A])
else:
    h_ch = chain_bwm[:,idx_A]
    
ax.hist2d(t0_ch, h_ch, **hist_kwargs)
ax.set_xlabel(r'$t_0$ (MJD)')
ax.set_ylabel(r'$\log_{10}(A_{BWM})$')
ax.set_ylim([-18,-8])

#fig.savefig("2Dpost_{}.pdf".format(psr_name))

In [ ]:
# de-log x-axis for ULs
if max(chain_bwm[:,idx_A])<0:
    chain_bwm[:,idx_A] = 10**chain_bwm[:,idx_A]
    params[idx_A] = r'$A_\mathrm{BwM}$'

UL, dUL = UL_uncert(chain_bwm[:,idx_A])
print("{}: {:.2e} +/- {:.2e}".format(ephem, UL, dUL))

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

# uses extracted chains from above
ch2d = np.vstack([t0_ch, h_ch])

Nbins = 20
tbins = np.linspace(t0_ch.min(), t0_ch.max(), Nbins+1)

data = np.zeros([Nbins,3])

for ii in range(Nbins):
    lbin, rbin = tbins[ii], tbins[ii+1]
    mask = (t0_ch<rbin)*(t0_ch>lbin)
    this_ch = ch2d[:,mask]
    this_mid = (lbin+rbin)/2
    this_UL, this_dUL = UL_uncert(10**this_ch[1], p=0.95)
    data[ii] = [this_mid, this_UL, this_dUL]

mids, ULs, dULs = data.T

ax.errorbar(mids, ULs, yerr=dULs, label=ephem)

ax.set_xlabel('$t_0$ (MJD)')
ax.set_ylabel('$h_{95\%}$')
ax.set_yscale('log')
ax.legend()

#fig.savefig('bwm-UL_v_t.pdf')